In [1]:
import pandas as pd
import numpy as np
import ipaddress as ip
from urllib.parse import urlparse
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tldextract

# Method to count number of dots
def countdots(url):  
    return url.count('.')

# Method to count number of delimeters
def countdelim(url):
    count = 0
    delim=[';','_','?','=','&']
    for each in url:
        if each in delim:
            count = count + 1
    
    return count

# Is IP addr present as th hostname, let's validate

import ipaddress as ip #works only in python 3

def is_ip(url):
    try:
        if ip.ip_address(url):
            return 1
    except:
        return 0


def isPresentDSlash(url):
    return url.count('//')

def countSubDir(url):
    return url.count('/')

def countSubDomain(subdomain):
    if not subdomain:
        return 0
    else:
        return len(subdomain.split('.'))

In [2]:
df = pd.read_csv("data.csv")
df_urls = df['url']
df_labels = df['label']

labels = []
for i in df_labels:
    if i == "good":
        labels.append(1)
    elif i == "bad":
        labels.append(0)
    else:
        printf("label error")
features = []
for url in df_urls:
    ext = tldextract.extract(url)
    features.append([countdots(url),len(url),countdelim(url),is_ip(url),isPresentDSlash(url), countSubDir(url), countSubDomain(ext.subdomain)])

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=0, stratify=labels)

# 数据预处理
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
    
#使用决策树算法
dt = DecisionTreeClassifier()
dt.fit(X_train_std, y_train)
print('Done fit')


#使用支持向量机算法,耗时会特别长。。。。
#clf = svm.SVC()
#clf.fit(X_train, y_train)
#print('Done fit')

Done fit


In [3]:
#分别计算准确率与召回率
from sklearn.metrics import precision_score, recall_score, accuracy_score
y_pred = dt.predict(X_test_std, y_test)
PRE = precision_score(y_test, y_pred)
ACC = accuracy_score(y_test, y_pred)
REC = recall_score(y_test, y_pred) 
print('Precision = %.2f  Accuracy = %.2f  Recall = %.2f' %(PRE, ACC, REC))

Precision = 0.87  Accuracy = 0.86  Recall = 0.97


In [4]:
# 输入单个恶意url判断其安全性

url = 'http://www.1cbc.com.cn'  #山寨工商银行
ext = tldextract.extract(url)
test_features = []
test_features.append([countdots(url),len(url),countdelim(url),is_ip(url),isPresentDSlash(url), countSubDir(url), countSubDomain(ext.subdomain)])
prediction = dt.predict(test_features)[0]

if prediction == 1:
    print(url + ' :bad url')
else:
    print(url + ' :good url')

http://www.1cbc.com.cn :bad url


In [5]:
# 输入单个正常url判断其安全性

url = 'http://edu.sc.gov.cn/'  #四川省教育厅
ext = tldextract.extract(url)
test_features = []
test_features.append([countdots(url),len(url),countdelim(url),is_ip(url),isPresentDSlash(url), countSubDir(url), countSubDomain(ext.subdomain)])
prediction = dt.predict(test_features)[0]

if prediction == 1:
    print(url + ' :good url')
else:
    print(url + ' :bad url')

http://edu.sc.gov.cn/ :good url
